<a href="https://colab.research.google.com/github/TThomas001/Python-for-Research-lecture-note/blob/main/Linear_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Optimization

- Model: LP, QP, SOCP, SDP, MIP, IP, MINLP, NLP...
  - categorized based on : `variables` `constraints` `objective` types
- Algorithms:gradient descent,simplex, interior point method...
  - need consider: run time | accuracy | memory 
- Solvers:CPLEX, Mosek, Gurobi, ECOS, Clp, Knitro, Lpopt
  - Implemntations of algorthms (libary)




# Intro to Linear Programing

This section use [Google OR-Tools] to solving optimization problem

resources:
- [Linear Programming Course](https://github.com/mlabonne/Linear-Programming-Course)
- [OR-Tools Guides](https://developers.google.com/optimization/introduction/python#setting_up_python)
- [Lecture note and case study](https://laurentlessard.com/teaching/524-intro-to-optimization/)

[Google OR-Tools]:https://developers.google.com/optimization


In [1]:
!python -m pip install --upgrade --user -q ortools
# restart runtime after installation

     |████████████████████████████████| 15.5 MB 22.6 MB/s 
     |████████████████████████████████| 407 kB 60.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
tensorflow-metadata 1.8.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.0 which is incompatible.


#Example Problme
<details><summary>Top Brass Trophy problem (Ex. 5.1 in Rardin'98)</summary>

Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: **football** and **soccer**. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns **\$12** in profit. Soccer trophies are similar except that a brass soccer ball is on top, and the unit profit is only **\$9**. Since the football has an asymmetric shape, its base requires **4 board feet of wood**; the soccer base requires only **2 board feet**. At the moment there are **1000 brass footballs in stock**, **1500 soccer balls**, **1750 plaques**, and **4800 board feet of wood**. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?
</details>

In [ ]:
from ortools.linear_solver import pywraplp
# Create a linear solver using the GLOP backend
solver = pywraplp.Solver('Top Brass Trophy problem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

- Decision variables: objective (parameters)
  - `NumVar` continous variables
  - `IntVar` interger vraibles
  - `BoolVar` boolean varaibles
- objective function: function of sum of cost and benefit (goal)
- constraints: reality rule (cost | limited resources)

In [ ]:
# create varaibles,set the limit
footabll_trophies = solver.IntVar(0, 1000, "footabll_trophies") 
soccer_trophies = solver.IntVar(0, 1500, "soccer_trophies")

# define constraints
solver.Add(4*footabll_trophies + 2*soccer_trophies <= 4800)
solver.Add(footabll_trophies + soccer_trophies <= 1750)

# declare objective
solver.Maximize(12*footabll_trophies + 9*soccer_trophies)

# Solve the problem
status = solver.Solve()

# output the result 
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print(f"Build {footabll_trophies.solution_value() } football trophies.")
  print(f"Build {soccer_trophies.solution_value() } soccer trophies.")
  print(f"Total profit will be ${solver.Objective().Value()}")

================= Solution =================
Solved in 89331.00 milliseconds in 2 iterations
Build 649.9999999999997 football trophies.
Build 1100.0000000000005 soccer trophies.
Total profit will be $17700.0


# matrix basic
- **matrix mutiplication**
  - require：colum and row the same
- **matrix transpons**: swaps row and columns
- **vector** is matrix n by 1 matrix
- vector multiplication
  - **inner(dot) product**:transpons first - vector produces a scalar
  - **outer product**:transpons second vector prodcut a $N\times M$ matix
-stack matrix:same dimensions can be stack and combines to bigger matrix
- **linear** and **affine** functions
  - linear `f(x)=Ax` 
    - $f(x_1, x_2, x_3... x_n)=a_1x_1+a_2x_2+a_3x_3...a_nx_n= a^Tx$
    - Geometry-[**hyperplane**](https://en.wikipedia.org/wiki/Arrangement_of_hyperplanes): vector a is noraml to hyperplane
    - Geometry-**subspace**: a setpoint satifing many linaer equation create intersection of many hpyerplanes
      - hyperplane = number of dimension -1
      - intersection k hyperplane dimension >= n-k
  - affine `f(x)=Ax+b` 
    - $f(x_1, x_2, x_3... x_n)=a_0 +a_1x_1+a_2x_2+a_3x_3...a_nx_n= a^Tx+b$
    - Geometry-**affine hyperplane**: shifted hyperplane
      - affine combination: combinations of poins in an affine subspace
      - Convex combinations: affine with constrain $0\le α\le1$
      - affine inequalities-[**halfspace**](https://inst.eecs.berkeley.edu/~ee127/sp21/livebook/l_lqp_half_spaces.html): $a^Tx\le b$, the intersection of halfsapce call **polyhedron**(polytope)
    - Geomery-**subspace**: a shifted subspace





##Transform to linear program

**stander form**: maximize $x∈ℜ^n$ $C^Tx$, subject to $Ax\le b$ and $x\geq 0$
<details><summary>tricks</summary>

  1. ocnverting min and max (negative)
      > $min(f(x)) == -max(-f(x))$
  2. reversing inequalities(flip sign)
      > $Ax\le b == (-A)x \geq b$
  3. equalities to inequalities(double up)
      > $ f(x) = 0 == f(x)\geq 0$ and $f(x)\le 0$
  4. inequalities to equalities(add slack)
      > $ f(x)\le 0 == f(x)+s = 0$ and $s\geq 0$
  5. unbounded to bouned(add difference)
      > $x∈ ℜ$ == $u \geq 0$, $v\geq 0$ and $x = u -v$
  6. bounded to nonegative(shift the variable)
      > $p\le x \le q$ == $0\le(x-p)$ and $x-p)≤(q-p)$ 
</details>

> linear program is optimziation model with:
  - real-valued vairbale $x ∈ ℜ^n$
  - affine objective function(min|max) $c^Tx+d$
  - constraints
    - affine equation $Ax=b$
    - affine inequalities $Ax \le b or Ax \geq b$
  - indivaule variables
    - box constraints ($ p \le x_i \le q$)
    - not constraints ($ x_i∈ ℜ$)
  - Solution of LP
    - **Infeasible**: no solution satifies all constraints
    - **unboundes**: solution exist, can be arbitryirly improve(missing constrain?) 
    - **on the boundary**: best solution found 

#Stander Fomr conversion
Conver the following linear porgram intor stander form:
$$\begin{aligned}\\\text{minimize}\qquad& p+q \\\text{subject to :} \qquad& 5p -3q =7 \\& 2p+1 \ge 2 \\& 1\le q\le 4 \end{aligned}$$

In [8]:
from ortools.linear_solver import pywraplp
# Create a linear solver using the GLOP backend
solver = pywraplp.Solver('LP problem example', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

# create varaibles,set the limit
p = solver.NumVar(-solver.infinity(),pywraplp.Solver_Infinity(),"p")
q = solver.IntVar(1,4,"q")

# define constraints
solver.Add(5*p -3*q == 7)
solver.Add(2*p+1>=2)

# declare objective
solver.Minimize(p+q)

# Solve the problem
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print(solver.ExportModelToProto)
  print('================= Solution =================')
  print(f'Solved in {solver.wall_time():.2f} milliseconds in {solver.iterations()} iterations')
  print(f"p = {p.solution_value()}")
  print(f"q = {q.solution_value()}")
  print(f"objective = {solver.Objective().Value()}")



<bound method Solver.ExportModelToProto of <ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x7f9b6aaaaed0> >>
================= Solution =================
Solved in 1.00 milliseconds in 0 iterations
p = 2.0
q = 1.0
objective = 3.0
